In [8]:
import ezdxf
import subprocess
import os
import glob
from collections import defaultdict, Counter
import ezdxf
import os
import geopandas as gpd
from shapely.geometry import LineString, Point
from scipy.spatial import cKDTree

Read dwg file and convert into dxf using ODA convertor 

In [ ]:
input_folder = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\dwg_file"
output_folder = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\dxf_file"
oda_exe = r"C:\Program Files\ODA\ODAFileConverter 26.7.0\ODAFileConverter.exe"

os.makedirs(output_folder, exist_ok=True)

cmd = [
    oda_exe,
    input_folder,
    output_folder,
    "ACAD2013",  # safer than 2010
    "DXF",
    "0",
    "*.dwg"
]

result = subprocess.run(cmd, capture_output=True, text=True)

print("Return code:", result.returncode)
print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)

dxf_files = glob.glob(os.path.join(output_folder, "*.dxf"))
print("Converted DXF files:", dxf_files)



def ODA_convertor(input_folder, output_folder, oda_exe):
    os.makedirs(output_folder, exist_ok=True)

    cmd = [
        oda_exe,
        input_folder,
        output_folder,
        "ACAD2013",  # safer than 2010
        "DXF",
        "0",
        "*.dwg"
    ]

    result = subprocess.run(cmd, capture_output=True, text=True)
    dxf_files = glob.glob(os.path.join(output_folder, "*.dxf"))

    print(dxf_files)

    return dxf_files





Return code: 0
STDOUT:
 
STDERR:
 
Converted DXF files: ['D:\\2_Analytics\\6_plan_vs_actual\\raw_data_dwg_file\\dxf_file\\ACW_NLM_Proposed_Pit_and_Dumping_area_for_FY_2024-25.dxf']


Read DXF file and split data into multiple layer and fetch section lines

In [ ]:
# === INPUT 
input_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\dxf_file\ACW_NLM_Proposed_Pit_and_Dumping_area_for_FY_2024-25.dxf"
output_folder = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_3"

def get_layers_from_dxf(input_dxf, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    out_file = os.path.join(output_folder, "section_lines.dxf")

    # === RULES ===
    # Only these types are allowed in the target "section lines" layer:
    ALLOWED_TYPES = {"LINE", "LWPOLYLINE", "TEXT", "MTEXT"}
    # Explicitly disallow common polygonal/fill types:
    DISALLOWED_TYPES = {"HATCH", "POLYLINE"}  # add more if needed (e.g., "SPLINE", "CIRCLE", ...)

    def is_lwpolyline_closed(lp) -> bool:
        """Return True if an LWPOLYLINE is closed."""
        # ezdxf LWPOLYLINE exposes .closed property in recent versions
        closed_attr = getattr(lp, "closed", None)
        if isinstance(closed_attr, bool):
            return closed_attr
        # Fallback: check DXF flags bit 1
        try:
            flags = int(lp.dxf.flags)
            return bool(flags & 1)
        except Exception:
            # If we can't determine, be conservative and treat as closed
            return True

    def layer_matches_section_criteria(entities):
        """Check that a layer contains ONLY allowed types, with at least one text and one line-ish entity,
        and that NO LWPOLYLINE is closed (i.e., no polygons)."""
        if not entities:
            return False

        type_counts = Counter(e.dxftype() for e in entities)

        # Hard disallow first
        if any(t in DISALLOWED_TYPES for t in type_counts):
            return False

        # Must be subset of allowed types
        if not set(type_counts).issubset(ALLOWED_TYPES):
            return False

        # Must have some text (section letters)
        text_count = type_counts.get("TEXT", 0) + type_counts.get("MTEXT", 0)
        if text_count == 0:
            return False

        # Must have some lines (LINE or LWPOLYLINE)
        lineish_count = type_counts.get("LINE", 0) + type_counts.get("LWPOLYLINE", 0)
        if lineish_count == 0:
            return False

        # Ensure no LWPOLYLINE is closed (to avoid polygons)
        for e in entities:
            if e.dxftype() == "LWPOLYLINE" and is_lwpolyline_closed(e):
                return False

        return True

    def save_entities_as_dxf(entities, filename):
        """Save given entities to a fresh DXF."""
        new_doc = ezdxf.new(setup=True)
        new_msp = new_doc.modelspace()
        for e in entities:
            try:
                new_msp.add_foreign_entity(e)
            except Exception as ex:
                print(f"  Skipped {e.dxftype()}: {ex}")
        new_doc.saveas(filename)

    # === LOAD & EVALUATE ===
    doc = ezdxf.readfile(input_dxf)
    msp = doc.modelspace()

    # Group entities by layer
    layer_entities = defaultdict(list)
    for ent in msp:
        layer_entities[ent.dxf.layer].append(ent)

    # Find candidate layers that satisfy criteria
    candidates = []
    for layer, ents in layer_entities.items():
        if layer_matches_section_criteria(ents):
            # Score by text count (prefer layer with most section letters)
            tcount = sum(1 for e in ents if e.dxftype() in ("TEXT", "MTEXT"))
            total = len(ents)
            candidates.append((layer, tcount, total, ents))

    if not candidates:
        print(" No layer matched the 'section lines + letters' criteria.")
    else:
        # Pick the best candidate: highest text count, then most entities as tiebreaker
        candidates.sort(key=lambda x: (x[1], x[2]), reverse=True)
        best_layer, best_texts, best_total, best_entities = candidates[0]

        # Save as section_lines.dxf (overwrite intentionally)
        save_entities_as_dxf(best_entities, out_file)

        # Console report
        print(f" Selected layer: '{best_layer}'")
        print(f"   - Text count: {best_texts}")
        print(f"   - Entity total: {best_total}")
        # Type breakdown for visibility
        type_counts = Counter(e.dxftype() for e in best_entities)
        for t, c in sorted(type_counts.items()):
            print(f"   - {t}: {c}")
        print(f" Saved as: {out_file}")

    return None


 Selected layer: 'SECTION LINE NAUKARI LST'
   - Text count: 36
   - Entity total: 55
   - LINE: 5
   - LWPOLYLINE: 14
   - MTEXT: 36
 Saved as: D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_3\section_lines.dxf


Remove duplicates from section line dxf file

In [7]:
# ---------------- Config ----------------
# Paths
section_lines_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_3\section_lines.dxf"
output_folder = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_3"
os.makedirs(output_folder, exist_ok=True)

# Tolerance via rounding precision (same approach you used)
PREC = 6  # decimals

# -------------- Helpers -----------------
def qr(val, p=PREC):
    return round(float(val), p)

def qpt3(v, p=PREC):
    # (x, y, z) with rounding
    if hasattr(v, "x"):
        return (qr(v.x, p), qr(v.y, p), qr(getattr(v, "z", 0.0), p))
    # assume tuple-like
    if len(v) == 2:
        return (qr(v[0], p), qr(v[1], p), 0.0)
    return (qr(v[0], p), qr(v[1], p), qr(v[2], p))

# ---------- Read source DXF ----------
doc_in = ezdxf.readfile(section_lines_dxf)
msp_in = doc_in.modelspace()

# ---------- Deduplicate GEOMETRY ----------
line_set = set()            # stores normalized segment tuples: ((x1,y1,z1),(x2,y2,z2))
text_geo_to_entity = {}     # key -> exemplar TEXT entity (keep first)
mtext_geo_to_entity = {}    # key -> exemplar MTEXT entity (keep first)

# ----- LINE entities -----
for line in msp_in.query("LINE"):
    start = qpt3(line.dxf.start)
    end = qpt3(line.dxf.end)
    key = tuple(sorted([start, end]))
    line_set.add(key)

# ----- LWPOLYLINE entities (as segments) -----
for pl in msp_in.query("LWPOLYLINE"):
    pts = list(pl.get_points("xy"))
    if len(pts) >= 2:
        for i in range(len(pts) - 1):
            a = (qr(pts[i][0]), qr(pts[i][1]), 0.0)
            b = (qr(pts[i+1][0]), qr(pts[i+1][1]), 0.0)
            key = tuple(sorted([a, b]))
            line_set.add(key)
        if pl.closed and len(pts) > 2:
            a = (qr(pts[-1][0]), qr(pts[-1][1]), 0.0)
            b = (qr(pts[0][0]), qr(pts[0][1]), 0.0)
            key = tuple(sorted([a, b]))
            line_set.add(key)

# ----- TEXT entities (geometry-only) -----
for tx in msp_in.query("TEXT"):
    ins = qpt3(tx.dxf.insert)
    h = qr(tx.dxf.height) if tx.dxf.hasattr("height") else 0.0
    rot = qr(tx.dxf.rotation) if tx.dxf.hasattr("rotation") else 0.0
    key = ("TEXT", ins, h, rot)
    # keep first seen by geometry
    if key not in text_geo_to_entity:
        text_geo_to_entity[key] = tx

# ----- MTEXT entities (geometry-only) -----
for mt in msp_in.query("MTEXT"):
    ins = qpt3(mt.dxf.insert)
    h = qr(mt.dxf.char_height) if mt.dxf.hasattr("char_height") else 0.0
    w = qr(mt.dxf.width) if mt.dxf.hasattr("width") else 0.0
    rot = qr(mt.dxf.rotation) if mt.dxf.hasattr("rotation") else 0.0
    key = ("MTEXT", ins, h, w, rot)
    if key not in mtext_geo_to_entity:
        mtext_geo_to_entity[key] = mt

print(f"Unique line segments: {len(line_set)}")
print(f"Unique TEXT by geometry: {len(text_geo_to_entity)}")
print(f"Unique MTEXT by geometry: {len(mtext_geo_to_entity)}")

# ---------- Write new DXF with uniques ----------
doc_out = ezdxf.new(setup=True)
msp_out = doc_out.modelspace()

# Optionally copy over layers first so created entities preserve original layers if you assign them.
# (ezdxf auto-creates layers on assignment, so this is optional.)

# Lines
for start, end in line_set:
    # msp.add_line accepts (x,y[,z]) tuples directly
    e = msp_out.add_line(start, end)

# TEXT – recreate using exemplar's geometry; content is from the first encountered entity
for key, tx in text_geo_to_entity.items():
    _, ins, h, rot = key
    # Create text; preserve layer for familiarity (not part of geometry, but safe to keep)
    new_tx = msp_out.add_text(tx.dxf.text or "", dxfattribs={
        "height": h,
        "rotation": rot,
        "layer": tx.dxf.layer if tx.dxf.hasattr("layer") else "0",
    })
    new_tx.set_pos(ins)  # sets insert (and aligns as baseline-left by default)

# MTEXT – recreate using exemplar's geometry; content from first encountered entity
for key, mt in mtext_geo_to_entity.items():
    _, ins, h, w, rot = key
    new_mt = msp_out.add_mtext(mt.text or "", dxfattribs={
        "char_height": h,
        "width": w,
        "rotation": rot,
        "layer": mt.dxf.layer if mt.dxf.hasattr("layer") else "0",
    })
    new_mt.set_location(ins)

out_path = os.path.join(output_folder, "section_lines_clean.dxf")
doc_out.saveas(out_path)
print("Cleaned DXF saved to:", out_path)


Unique line segments: 18
Unique TEXT by geometry: 0
Unique MTEXT by geometry: 36
Cleaned DXF saved to: D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_3\section_lines_clean.dxf


Assign the text to each line

In [9]:
# ---------- Input & Output ----------
input_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_3\section_lines_clean.dxf"
work_dir = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_3\processed"
os.makedirs(work_dir, exist_ok=True)

lines_dxf = os.path.join(work_dir, "LINES.dxf")
texts_dxf = os.path.join(work_dir, "TEXTS.dxf")
output_shp = os.path.join(work_dir, "LINES_with_start_end_text.shp")

# ---------- Step 1: Split DXF ----------
doc = ezdxf.readfile(input_dxf)
msp = doc.modelspace()

line_entities, text_entities = [], []
for e in msp:
    etype = e.dxftype()
    if etype in ["LINE", "LWPOLYLINE"]:
        line_entities.append(e)
    elif etype in ["TEXT", "MTEXT"]:
        text_entities.append(e)

# Save lines
if line_entities:
    doc_lines = ezdxf.new(setup=True)
    msp_lines = doc_lines.modelspace()
    for e in line_entities:
        try:
            msp_lines.add_foreign_entity(e)
        except Exception as ex:
            print(f" Skipped line entity: {ex}")
    doc_lines.saveas(lines_dxf)

# Save texts
if text_entities:
    doc_texts = ezdxf.new(setup=True)
    msp_texts = doc_texts.modelspace()
    for e in text_entities:
        try:
            msp_texts.add_foreign_entity(e)
        except Exception as ex:
            print(f" Skipped text entity: {ex}")
    doc_texts.saveas(texts_dxf)

print(f"Split complete: {len(line_entities)} lines, {len(text_entities)} texts")

# ---------- Step 2: Convert Lines to GeoDataFrame ----------
doc_lines = ezdxf.readfile(lines_dxf)
msp_lines = doc_lines.modelspace()

lines = []
for line in msp_lines.query("LINE"):
    start = (line.dxf.start.x, line.dxf.start.y)
    end = (line.dxf.end.x, line.dxf.end.y)
    lines.append({"geometry": LineString([start, end]), "start": start, "end": end})

for pline in msp_lines.query("LWPOLYLINE"):
    pts = list(pline.get_points("xy"))
    if len(pts) > 1:
        for i in range(len(pts) - 1):
            start, end = pts[i], pts[i+1]
            lines.append({"geometry": LineString([start, end]), "start": start, "end": end})
        if pline.closed:
            start, end = pts[-1], pts[0]
            lines.append({"geometry": LineString([start, end]), "start": start, "end": end})

gdf_lines = gpd.GeoDataFrame(lines, crs="EPSG:32644")  
gdf_lines = gdf_lines.drop_duplicates(subset=["geometry"])
print(f"Deduplicated lines: {len(gdf_lines)}")

# ---------- Step 3: Convert Texts to GeoDataFrame ----------
doc_texts = ezdxf.readfile(texts_dxf)
msp_texts = doc_texts.modelspace()

texts = []
for t in msp_texts.query("TEXT MTEXT"):
    content = t.dxf.text.strip()
    insert = (t.dxf.insert.x, t.dxf.insert.y)
    texts.append({"geometry": Point(insert), "text": content})

gdf_texts = gpd.GeoDataFrame(texts, crs=gdf_lines.crs)
gdf_texts = gdf_texts.drop_duplicates(subset=["geometry", "text"])
print(f" Deduplicated texts: {len(gdf_texts)}")

# ---------- Step 4: Nearest text for start & end ----------
tree = cKDTree(gdf_texts.geometry.apply(lambda p: (p.x, p.y)).to_list())

def nearest_text(point):
    dist, idx = tree.query([point.x, point.y], k=1)
    return gdf_texts.iloc[idx]["text"]

start_labels, end_labels = [], []
for _, row in gdf_lines.iterrows():
    start_point = Point(row["start"])
    end_point = Point(row["end"])
    start_labels.append(nearest_text(start_point))
    end_labels.append(nearest_text(end_point))

gdf_lines["start_text"] = start_labels
gdf_lines["end_text"] = end_labels

# ---------- Step 5: Save Shapefile ----------
gdf_lines.to_file(output_shp)
print(f" Final line shapefile with start/end texts saved: {output_shp}")

Split complete: 18 lines, 36 texts
Deduplicated lines: 18
 Deduplicated texts: 36
 Final line shapefile with start/end texts saved: D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_3\processed\LINES_with_start_end_text.shp
